# Voronoi Analysis of Atom Positions
## Part of the **pycroscopy** package
### Gerd Duscher and Rama
University of Tennessee, Knoxville and ORNL <br> 
10/03/2021

#### This Jupyter Notebook demonstrates how to use Pycroscopy to find atom positions in images and subsequenitally analayse them with Voronoi tesselation.

This notebook uses a lot of the image packages of pycroscopy to clean images, find atoms, and to analyse the positions.

The atomic positions are viewed as a graph; more specifically a ring graph.
The projections of the unit_cell and defect structural units are the rings.
The center of the ring is the interstitial location. 
The vertices are the atom positions and the edges are the bonds.

This is a modification of the method of [Banadaki and Patala](http://dx.doi.org/10.1038/s41524-017-0016-0)
for 2-D (works in 3D as well)

Starting from the atom positions we make a Delaunay tesselation and determine the size of the intersitital (circumscribed circle radius minus the atom radius).

If neighbouring interstitials overlap we merge those triangles (in 3D the tetrhedra). This will give an unanbiguous tesselation or graph for a given atomic size.


![notebook_rules.png](https://raw.githubusercontent.com/pycroscopy/pycroscopy/master/jupyter_notebooks/notebook_rules.png)


In [ ]:
# Make sure needed packages are installed and up-to-date
import sys
!conda install --yes --prefix {sys.prefix} numpy scipy matplotlib Ipython ipywidgets SciFiReaders
!{sys.executable} -m pip install -U --no-deps pycroscopy  # this will automatically install sidpy as well

In [15]:
%pylab widget
import sys

sys.path.insert(0, '../')

from sidpy.io.interface_utils import open_file_dialog
print('sidpy version: ', sidpy.__version__)

from SciFiReaders import DM3Reader
import SciFiReaders
print('SciFiReaders version: ', SciFiReaders.__version__)

%load_ext autoreload
%autoreload 2
import pycroscopy as px

Populating the interactive namespace from numpy and matplotlib
sidpy version:  0.0.6
SciFiReaders version:  0.0.1
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Image

In [2]:
dialog = open_file_dialog()
dialog

FileChooser(path='.', filename='', title='HTML(value='', layout=Layout(display='none'))', show_hidden='False',…

In [17]:
dm3_reader = DM3Reader(dialog.selected)
dataset = dm3_reader.read()
pixel_size = 16/512*0.97
dataset.set_dimension(0, sidpy.Dimension(np.arange(dataset.shape[0])*pixel_size,
                                              name='x', units='nm', dimension_type='SPATIAL',
                                              quantity='length'))
dataset.set_dimension(1, sidpy.Dimension(np.arange(dataset.shape[1])*pixel_size,
                                              name='y', units='nm', dimension_type='SPATIAL',
                                              quantity='length'))

dataset.plot()
from matplotlib.widgets import RectangleSelector
# drawtype is 'box' or 'line' or 'none'
selector = RectangleSelector(plt.gca(), None ,drawtype='box', useblit=True,
                                       button=[1, 3],  # disable middle button
                                       minspanx=5, minspany=5,
                                       spancoords='pixels',
                                       interactive=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
selection = px.image.crop_image(dataset, selector.corners)
selection.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Clean Image

In [19]:
# ----Input --------------
resolution = 0.05  # in nm
# ------------------------

lr_dset = px.image.decon_lr(dataset-dataset.min(), resolution=resolution)

lr_dset[lr_dset>12] = 12
lr_dset.plot()

  0%|          | 0/500 [00:00<?, ?it/s]

converged in 252 iterations

 Lucy-Richardson deconvolution converged in 252  iterations


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
svd_dset = px.image.clean_svd(dataset, source_size=3)
svd_dset.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Find Atoms
### Blob Finder with Pixel Accuracy

In [6]:
# ----Input ---------
atom_size = 0.1  # units of image scale
threshold = 0.03
# -------------------
atoms = px.image.find_atoms(lr_dset, atom_size=atom_size, threshold=threshold)
print(f' Found {len(atoms)} atoms in image')

plt.figure()
plt.imshow(lr_dset.T, vmax=9)
plt.scatter(atoms[:, 0], atoms[:,1], color='red')

 Found 7632 atoms in image


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Refine Atom Positions 
Refine atom positions  with pixel sub-pixel accuracy by fitting a (symmetric) Gaussian in peak

In [7]:
sym = px.image.atom_refine(np.array(lr_dset-lr_dset.min()+1e-12), atoms, 2, max_int = 0, min_int = 2, max_dist = 2)
refined_atoms = np.array(sym['atoms'])

plt.figure()
plt.imshow(lr_dset.T, vmax=9)
plt.scatter(refined_atoms[:, 0]+0.5, refined_atoms[:,1]+0.5, color='red')

using radius  2 pixels


  0%|          | 0/7632 [00:00<?, ?it/s]

/home/nyd/.local/lib/python3.8/site-packages/pyTEMlib/probe_tools.py:13: RuntimeWarning: invalid value encountered in true_divide
  probe = g / g.sum() * intensity


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 

In [17]:
projected(.404/np.sqrt(2)/4)

0.07071067811865475

In [8]:
structural_units  = px.image.find_structural_units(refined_atoms[:,:2], .4/np.sqrt(2)/4, lr_dset)

graph_dictionary = px.image.get_polygons(structural_units)

fig = plt.figure()
plt.imshow(lr_dset.T, extent=[-0.5,dataset.shape[0]-1.5, dataset.shape[1]-1.5,-0.5], cmap = 'gray', vmax= 7)

px.image.add_graph(graph_dictionary, 'cyclicity', min_q=2.5, max_q=12.5, fig=fig, cmap=plt.cm.tab10)
# px.image.add_graph(graph_dictionary, 'areas', min_q=4**2, max_q=8**2, fig=fig)

  0%|          | 0/6351 [00:00<?, ?it/s]

  0%|          | 0/14899 [00:00<?, ?it/s]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
from matplotlib.collections import PatchCollection
import matplotlib

unit_cells = PatchCollection(graph_dictionary['unit_cells'], alpha=.5, cmap=matplotlib.cm.viridis, clim=(4., 8.),  edgecolor='black')
cyclicity = np.sqrt(np.array(graph_dictionary['areas']))

plt.figure()
plt.imshow(lr_dset.T, extent=[-0.5,dataset.shape[0]-1.5, dataset.shape[1]-1.5,-0.5], cmap='gray', vmax= 5, vmin = 0)

unit_cells.set_array(cyclicity)
plt.gca().add_collection(unit_cells)
plt.scatter(refined_atoms[:,0],refined_atoms[:,1],color='orange',alpha=0.5, s = 20)

cbar = plt.colorbar(unit_cells, label='$\sqrt{area}$ [nm]')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
import scipy
vor = scipy.spatial.Voronoi(refined_atoms[:,:2])
fig = scipy.spatial.voronoi_plot_2d(vor)
plt.gca().imshow(lr_dset.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Appendix

In [260]:
from tqdm.auto import trange, tqdm

def circum_center(vertex_pos, tol=1e-3):
    """
    Function finds the center and the radius of the circumsphere of every tetrahedron.
    Reference:
    Fiedler, Miroslav. Matrices and graphs in geometry. No. 139. Cambridge University Press, 2011.
    (p.29 bottom: example 2.1.11)
    Code (slightly modified) from https://github.com/spatala/gbpy

    Parameters
    -----------------
    vertex_pos : numpy array
        The position of vertices of a tetrahedron
    tol : float
        Tolerance defined  to identify co-planar tetrahedrons
    Returns
    ----------
    circum_center : numpy array
        The center of the circumsphere
    circum_radius : float
        The radius of the circumsphere
    """
    
    if vertex_pos.shape[1] < 3:
        ax = vertex_pos[0, 0]
        ay = vertex_pos[0, 1]
        bx = vertex_pos[1, 0]
        by = vertex_pos[1, 1]
        cx = vertex_pos[2, 0]
        cy = vertex_pos[2, 1]
        d = 2 * (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by))
        ux = ((ax * ax + ay * ay) * (by - cy) + (bx * bx + by * by) * (cy - ay) + (cx * cx + cy * cy) * (ay - by)) / d
        uy = ((ax * ax + ay * ay) * (cx - bx) + (bx * bx + by * by) * (ax - cx) + (cx * cx + cy * cy) * (bx - ax)) / d

        circum_center =np.array([ux, uy]) 
        circum_radius = np.linalg.norm(circum_center-vertex_pos[0])
        
        return np.array(circum_center), circum_radius
    dis_ij = scipy.spatial.distance.pdist(np.array(vertex_pos), 'euclidean')
    sq_12, sq_13, sq_14, sq_23, sq_24, sq_34 = np.power(dis_ij, 2)

    matrix_c = np.array([[0, 1, 1, 1, 1], [1, 0, sq_12, sq_13, sq_14], [1, sq_12, 0, sq_23, sq_24],
                         [1, sq_13, sq_23, 0, sq_34], [1, sq_14, sq_24, sq_34, 0]])

    det_matrix_c = (np.linalg.det(matrix_c))

    if det_matrix_c < tol:
        return np.array([0, 0, 0]), 0
    else:
        matrix = -2 * np.linalg.inv(matrix_c)
        circum_center = (matrix[0, 1] * vertex_pos[0, :] + matrix[0, 2] * vertex_pos[1, :] +
                         matrix[0, 3] * vertex_pos[2, :] +
                         matrix[0, 4] * vertex_pos[3, :]) / (matrix[0, 1] + matrix[0, 2] + matrix[0, 3] + matrix[0, 4])
        circum_radius = np.sqrt(matrix[0, 0]) / 2

    return np.array(circum_center), circum_radius

def voronoi_volumes(points):
    """
    Volumes of voronoi  cells from
    https://stackoverflow.com/questions/19634993/volume-of-voronoi-cell-python


    """
    v = scipy.spatial.Voronoi(points)
    vol = np.zeros(v.npoints)
    for i, reg_num in enumerate(v.point_region):
        indices = v.regions[reg_num]
        if -1 in indices: # some regions can be opened
            vol[i] = np.inf
        else:
            try:
                hull = scipy.spatial.ConvexHull(v.vertices[indices])
                vol[i] = hull.volume
            except:
                vol[i] = 0.
    return vol


def get_voronoi(tetrahedra, atoms, r_a, extent):
    """
    Find Voronoi vertices and keep track of associated tetrahedrons and interstitial radii
    
    Used in find_polyhedra function
    
    Parameters
    ----------
    tetrahedra: scipy.spatial.Delaunay object
        Delaunay tesselation
    atoms: ase.Atoms object
        the structural information
    r_a: float
        the atomic radius

    Returns
    -------
    voronoi_vertices: list
        list of positions of voronoi vertices
    voronoi_tetrahedra:
        list of indices of associated vertices of tetrahedra
    r_vv: list of float
        list of all interstitial sizes
    """

    voronoi_vertices = []
    voronoi_tetrahedrons = []
    r_vv = []
    for vertices in tetrahedra.vertices:
        voronoi, radius = circum_center(atoms[vertices])
        
        if (voronoi >= 0).all() and (extent - voronoi > 0).all() and radius > 0.01:
            voronoi_vertices.append(voronoi)
            voronoi_tetrahedrons.append(vertices)
            r_vv.append(radius - r_a)
    return voronoi_vertices, voronoi_tetrahedrons, r_vv


def find_overlapping_interstitials(voronoi_vertices, r_vv, r_a, cheat=1.):
    """Find overlapping spheres"""
    
    vertex_tree = scipy.spatial.cKDTree(np.array(voronoi_vertices)[:,:2])

    pairs = vertex_tree.query_pairs(r=r_a * 2)

    overlapping_pairs = []
    for (i, j) in pairs:
        if np.linalg.norm(voronoi_vertices[i] - voronoi_vertices[j]) < (r_vv[i] + r_vv[j])*cheat:
            overlapping_pairs.append([i, j])

    return np.array(sorted(overlapping_pairs))


def find_clusters(overlapping_pairs):
    """Make cluste
    We are using a breadth first to go through the list of overlapping spheres to determine clusters
    """
    visited_all = []
    clusters = []
    for initial in overlapping_pairs[:, 0]:
        if initial not in visited_all:
            # breadth first search
            visited = []  # the atoms we visited
            queue = [initial]
            while queue:
                node = queue.pop(0)
                if node not in visited_all:
                    visited.append(node)
                    visited_all.append(node)
                    # neighbors = overlapping_pairs[overlapping_pairs[:,0]==node,1]
                    neighbors = np.append(overlapping_pairs[overlapping_pairs[:, 1] == node, 0],
                                          overlapping_pairs[overlapping_pairs[:, 0] == node, 1])

                    for i, neighbour in enumerate(neighbors):
                        if neighbour not in visited:
                            queue.append(neighbour)
            clusters.append(visited)
    return clusters, visited_all


def make_polyhedrons(atoms, voronoi_vertices, voronoi_tetrahedrons, clusters, visited_all):
    """collect output data  and make dictionary"""

    polyhedra = {}
    for index in trange(len(clusters)):
        cluster = clusters[index]
        cc = []
        for c in cluster:
            cc = cc + list(voronoi_tetrahedrons[c])
        hull = scipy.spatial.ConvexHull(atoms[list(set(cc))])
        faces = []
        triangles = []
        for s in hull.simplices:
            faces.append(atoms[list(set(cc))][s])
            triangles.append(list(s))
        polyhedra[index] = {'vertices': atoms[list(set(cc))], 'indices': list(set(cc)),
                            'faces': faces, 'triangles': triangles,
                            'length': len(list(set(cc))),
                            'combined_vertices': cluster,
                            'interstitial_index': index,
                            'interstitial_site': np.array(voronoi_tetrahedrons)[cluster].mean(axis=0),
                            'volume': hull.volume}
        if False:  # isinstance(atoms, ase.Atoms):
                polyhedra[index]['atomic_numbers'] = atoms.get_atomic_numbers()[vertices],

        # 'coplanar': hull.coplanar}

    running_number = index + 0
    for index in trange(len(voronoi_vertices)):
        if index not in visited_all:
            vertices = voronoi_tetrahedrons[index]
            hull = scipy.spatial.ConvexHull(atoms[vertices])
            faces = []
            triangles = []
            for s in hull.simplices:
                faces.append(atoms[vertices][s])
                triangles.append(list(s))

            polyhedra[running_number] = {'vertices': atoms[vertices], 'indices': vertices,
                                         'faces': faces, 'triangles': triangles,
                                         'length': len(vertices),
                                         'combined_vertices': index,
                                         'interstitial_index': running_number,
                                         'interstitial_site': np.array(voronoi_tetrahedrons)[index],
                                         'volume': hull.volume}
            if False:  # isinstance(atoms, ase.Atoms):
                polyhedra[running_number]['atomic_numbers'] = atoms.get_atomic_numbers()[vertices],


            running_number += 1

    return polyhedra

##################################################################
# polyhedra functions
##################################################################


def find_polyhedra(atoms, r_a, extent, cheat=1.0):
    """ get polyhedra information from an ase.Atoms object

    This is following the method of Banadaki and Patala
    http://dx.doi.org/10.1038/s41524-017-0016-0

    Parameter
    ---------
    atoms: ase.Atoms object
        the structural information
    r_a: float
        the atomic radius

    Returns
    -------
    polyhedra: dict
        dictionary with all information of polyhedra
    """
    
    if not isinstance(r_a, (int, float)):
        raise TypeError('Atomic radius must be a real number')

    if not (0.5 < r_a < 2):
        print('Strange atomic radius, are you sure you know what you are doing?')
    tesselation = scipy.spatial.Delaunay(atoms)

    voronoi_vertices, voronoi_tetrahedrons, r_vv = get_voronoi(tesselation, atoms, r_a, extent)

    overlapping_pairs = find_overlapping_interstitials(voronoi_vertices, r_vv, r_a, cheat=cheat)

    clusters, visited_all = find_clusters(overlapping_pairs)

    polyhedra = make_polyhedrons(atoms, voronoi_vertices, voronoi_tetrahedrons, clusters, visited_all)

    return polyhedra


def sort_polyhedra_by_vertices(polyhedra, visible=range(4, 100), z_lim=[0, 100], verbose=False):
    indices = []

    for key, polyhedron in polyhedra.items():
        if 'length' not in polyhedron:
            polyhedron['length'] = len(polyhedron['vertices'])

        if polyhedron['length'] in visible:
            center = polyhedron['vertices'].mean(axis=0)
            if z_lim[0] < center[2] < z_lim[1]:
                indices.append(key)
                if verbose:
                    print(key, polyhedron['length'], center)
    return indices


# color_scheme = ['lightyellow', 'silver', 'rosybrown', 'lightsteelblue', 'orange', 'cyan', 'blue', 'magenta',
#                'firebrick', 'forestgreen']

In [235]:
polyhedra  = find_polyhedra(atoms[:,:2], 4, extent=[lr_dset.shape[0],lr_dset.shape[1]])

Strange atomic radius, are you sure you know what you are doing?


In [286]:
import matplotlib.patches as patches

def get_poly(polyhedra):
    """ Make graph from atom positions

    Parameters
    ----------
    atoms: numpy array (nx2)
        positions of atoms to be evaluated for graph
    extent: list of float (4x1)
        extent of image
    smallest_lattice_parameter: float
        determines how far the Voronoi vertices have to be apart to be considered a distortion

    Returns
    -------
    tags: dictionary
        information of graph
    """

    rings = []
    centers = []
    _inner_angles = []
    cyclicities = []
    cells = []
    areas = []
    for key, poly in polyhedra.items():
        corners = poly['vertices']
        if len(corners) > 2:
            cyclicities.append(len(corners))  # length of ring or cyclicity will be stored
            center = np.average(corners, axis=0)  # center of ring will be stored
            centers.append(center)
            angles = np.arctan2(corners[:, 1] - center[1], corners[:, 0] - center[0])
            ang_sort = np.argsort(angles)
            angles = (angles[ang_sort] - angles[np.roll(ang_sort, 1)]) % np.pi
            _inner_angles.append(angles)  # inner angles in radians

            ring = corners[ang_sort]  # clocks=wise sorted ring vertices will be stored
            rings.append(ring)
            areas.append(poly['volume'])
            cells.append(patches.Polygon(ring, closed=True, fill=True, edgecolor='red', linewidth=2))

    max_ring_size = max(cyclicities)
    tags = {'unit_cells': cells, 'centers': np.array(centers), 'cyclicity': np.array(cyclicities), 'areas': np.array(areas)}

    number_of_rings = len(rings)
    tags['vertices'] = np.zeros((number_of_rings, max_ring_size, 2))
    tags['inner_angles'] = np.zeros((number_of_rings, max_ring_size))
    tags['areas'] = areas

    # a slow way to make a sparse matrix, which we need for h5_file
    for i in range(number_of_rings):
        ring = rings[i]
        angles = _inner_angles[i]
        tags['vertices'][i, :len(ring), :] = ring
        tags['inner_angles'][i, :len(ring)] = angles

    return tags
graph_dictionary = get_poly(polyhedra)

In [312]:
from matplotlib.collections import PatchCollection
from matplotlib import cm
import matplotlib

unit_cells = PatchCollection(graph_dictionary['unit_cells'], alpha=1.,  cmap=matplotlib.cm.Blues)
cyclicity = np.array(graph_dictionary['cyclicity'])


plt.figure()
# plt.imshow(lr_dset.T, cmap='gray', vmax = 8)

unit_cells.set_array(cyclicity)
plt.gca().add_collection(unit_cells)
#plt.scatter(centers[:,0],centers[:,1],color='blue',alpha=0.5, s = 3)

cbar = plt.colorbar(unit_cells, label='cyclicity')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [305]:
np.array(cyclicity).max()

AttributeError: 'numpy.ndarray' object has no attribute 'median'

In [285]:
plt.figure()
#plt.title('area of ' + main_dataset.title)
plt.imshow(lr_dset.T, cmap = 'gray')

unit_cells.set_array(np.sqrt(graph_dictionary['areas']))

plt.gca().add_collection(unit_cells)
plt.scatter(centers[:,0],centers[:,1],color='blue',alpha=0.5, s=2)

cbar = plt.colorbar(unit_cells, label='$\sqrt{area}$ [nm]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

RuntimeError: Can not put single artist in more than one figure

In [227]:
polyhedra[37]
poly = polyhedra[44]

plt.figure()
plt.imshow(lr_dset.T, vmax=9)
plt.scatter(poly['vertices'][:, 0], poly['vertices'][:,1], color='red')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [192]:
##################################################################
# plotting functions
##################################################################


def plot_super_cell(super_cell, shift_x=0):
    """ make a super_cell to plot with extra atoms at periodic boundaries"""

    if not isinstance(super_cell, ase.Atoms):
        raise TypeError('Need an ase Atoms object')

    plot_boundary = super_cell * (2, 2, 3)
    plot_boundary.positions[:, 0] = plot_boundary.positions[:, 0] - super_cell.cell[0, 0] * shift_x

    del plot_boundary[plot_boundary.positions[:, 2] > super_cell.cell[2, 2] * 1.5 + 0.1]
    del plot_boundary[plot_boundary.positions[:, 1] > super_cell.cell[1, 1] + 0.1]
    del plot_boundary[plot_boundary.positions[:, 0] > super_cell.cell[0, 0] + 0.1]
    del plot_boundary[plot_boundary.positions[:, 0] < -0.1]
    plot_boundary.cell = super_cell.cell * (1, 1, 1.5)

    return plot_boundary


def plot_polyhedron(polyhedra, indices, center=False):
    if isinstance(indices, int):
        indices = [indices]
    if len(indices) == 0:
        print('Did not find any polyhedra')
        return {}

    center_point = np.mean(polyhedra[indices[0]]['vertices'], axis=0)

    if center:
        print(center_point)
        center = center_point
    else:
        center = [0, 0, 0]

    data = []
    for index in indices:
        polyhedron = polyhedra[index]

        vertices = polyhedron['vertices'] - center
        faces = np.array(polyhedron['triangles'])
        x, y, z = vertices.T
        i_i, j_j, k_k = faces.T

        mesh = dict(type='mesh3d',
                    x=x,
                    y=y,
                    z=z,
                    i=i_i,
                    j=j_j,
                    k=k_k,
                    name='',
                    opacity=0.2,
                    color=px.colors.qualitative.Light24[len(vertices) % 24]
                    )
        tri_vertices = vertices[faces]
        x_e = []
        y_e = []
        z_e = []
        for t_v in tri_vertices:
            x_e += [t_v[k % 3][0] for k in range(4)] + [None]
            y_e += [t_v[k % 3][1] for k in range(4)] + [None]
            z_e += [t_v[k % 3][2] for k in range(4)] + [None]

        # define the lines to be plotted
        lines = dict(type='scatter3d',
                     x=x_e,
                     y=y_e,
                     z=z_e,
                     mode='lines',
                     name='',
                     line=dict(color='rgb(70,70,70)', width=1.5))
        data.append(mesh)
        data.append(lines)
    return data

In [54]:
def make_new_vertices(vertices, extent, smallest_lattice_parameter):
    """ Determine whether vertices are too close and have to be replaced by median

    Part of get_graph function
    Parameters
    ----------
    vertices: numpy array (nx2)
        vertices of Voronoi tiles to be evaluated
    extent: list of float (4x1)
        extent of image
    smallest_lattice_parameter: float
        determines how far the Voronoi vertices have to be apart to be considered caused by distortion

    Returns
    -------
    new_voronoi: numpy array
        vertices of new Voronoi tiling
    """

    vertices_tree = scipy.spatial.cKDTree(vertices)

    dis = vertices_tree.query_ball_point(vertices, r=smallest_lattice_parameter * .7, p=2)  # , return_length=True)
    nn = vertices_tree.query_ball_point(vertices, r=smallest_lattice_parameter * .7, p=2, return_length=True)

    # handle nn > 2 differently Gerd

    new_voronoi = []
    for near in dis:
        if len(near) > 1:
            new = np.average(vertices[near], axis=0)
        elif len(near) > 0:
            new = vertices[near][0]
        else:
            new = [-1, -1]

        if (new > 0).all() and (new[0] < extent[1]) and (new[1] < extent[2]):
            new_voronoi.append(new)

    ver_sort = np.argsort(nn)
    nn_now = nn[ver_sort[-1]]
    done_list = []
    i = 1
    while nn_now > 2:
        close_vertices = dis[ver_sort[-i]]
        new_vert = []

        for vert in close_vertices:
            if vert not in done_list:
                new_vert.append(vert)

        done_list.extend(new_vert)
        # check whether necessary big_vertex = np.average(vertices[new_vert], axis=0)
        if len(new_vert) > 1:
            big_vertex = np.average(vertices[new_vert], axis=0)
            if (big_vertex[0] > 0) and (big_vertex[1] > 0):
                new_voronoi.append([big_vertex[0], big_vertex[1]])
        elif len(new_vert) > 0:
            new_voronoi.append([(vertices[new_vert[0]])[0], (vertices[new_vert[0]])[1]])
        i += 1
        nn_now = nn[ver_sort[-i]]

    # print(len(new_voronoi))
    new_voronoi = np.unique(new_voronoi, axis=0)
    return new_voronoi